In [ ]:
!pip install transformers 
!pip install tokenizers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
     |████████████████████████████████| 56 kB 2.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import transformers
from tqdm import tqdm
from collections import namedtuple
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import  matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from tokenizers import BertWordPieceTokenizer

import warnings 
warnings.simplefilter('ignore')

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.92.137.10:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.92.137.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.92.137.10:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
train

,OriginalTweet,Sentiment
0,advice talk to your neighbours family to excha...,4.0
1,coronavirus australia woolworths to give elder...,4.0
2,my food stock is not the only one which is emp...,4.0
3,me ready to go at supermarket during the covid...,1.0
4,as news of the region first confirmed covid ca...,4.0
...,...,...
28352,been to chemist to buy some meds and they stri...,2.0
28353,and this afternoon groceries went to korean su...,4.0
28354,if am being encouraged to wear bandana when go...,4.0
28355,need sugar daddy fast cause this covid thing i...,5.0


In [ ]:
clean_data=pd.read_csv('./clean_data.csv')
clean_data.drop(['Unnamed: 0','Sequence_length'],axis=1,inplace=True)

In [ ]:
train=clean_data.loc[:40000]
test=clean_data.loc[40000:]

In [ ]:
train

,OriginalTweet,Sentiment
0,advice talk to your neighbours family to excha...,4
1,coronavirus australia woolworths to give elder...,4
2,my food stock is not the only one which is emp...,4
3,me ready to go at supermarket during the covid...,1
4,as news of the region first confirmed covid ca...,4
...,...,...
39996,you better cut down data prices we got another...,4
39997,do not buy any of smithfield meat because they...,2
39998,there no authorized home testing kit so essent...,1
39999,uranium enters bull market after covid hits su...,2


In [ ]:
tokenizer=transformers.DistilBertTokenizer.from_pretrained('bert-base-uncased')
texts=[x for x in train.OriginalTweet]
def regular_encode(texts,tokenizer=tokenizer,max_len=50):
    reg_token=tokenizer.batch_encode_plus(batch_text_or_text_pairs=texts,
                                          add_special_tokens=False,
                                          padding=True,
                                          max_length=max_len,truncation=True)
    return np.array(reg_token['input_ids'])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [ ]:
def build_model(transformer,batch_size=32,max_len=50):
    Input_layer=Input(shape=(max_len,),dtype=tf.int32,name='Input_layer')
    sequence_output=transformer(Input_layer)[0]
    cls_token=sequence_output[:,0,:]
    out=Dense(5,activation='softmax')(cls_token)
    model=Model(Input_layer,outputs=out)
    model.compile(Adam(learning_rate=6e-6),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model


In [ ]:
X=regular_encode(texts=texts)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(clean_data.Sentiment)
Y=le.transform(train.Sentiment)

In [ ]:
AUTO=tf.data.experimental.AUTOTUNE
def create_train(X,Y):
    return tf.data.Dataset.from_tensor_slices((X,Y)).repeat().shuffle(42).batch(32).prefetch(AUTO)

In [ ]:
def create_validation(X,Y):
    return tf.data.Dataset.from_tensor_slices((X,Y)).batch(32).cache().prefetch(AUTO)

In [ ]:
X_test=regular_encode(texts=[x for x in test.OriginalTweet])
Y_test=le.transform(test.Sentiment)

In [ ]:
val_preds=np.zeros((train.shape[0],5))
test_preds=[]
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import KFold
Folds=5
skf=KFold(n_splits=Folds,shuffle=True,random_state=42)
early_stopping=EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_accuracy', patience = 2)
for fold,(trn_index,val_index) in enumerate(skf.split(X,Y)):
    print('fold: ',fold+1)
    print('-'*50)
    X_train,Y_train=X[trn_index,:],Y[trn_index]
    X_val,Y_val=X[val_index,:],Y[val_index]
    with strategy.scope():
        transformer_layer=transformers.TFDistilBertModel.from_pretrained('bert-base-uncased')
        model=build_model(transformer_layer,32)
    model_checkpoint=ModelCheckpoint(filepath='fold-%i.h5'%fold,save_best_only=True,verbose=0,save_weights_only=True,mode='max',save_freq='epoch')
    callbacks=[model_checkpoint,early_stopping]
    history=model.fit(create_train(X_train,Y_train),batch_size=32,steps_per_epoch=32000//32,
                      validation_data=create_validation(X_val,Y_val),epochs=7,
                      validation_batch_size=32,callbacks=callbacks)
    val_preds[val_index]=model.predict(X_val)
    test_preds.append(model.predict(X_test))




fold:  1
--------------------------------------------------


You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFDistilBertModel: ['bert', 'nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['distilbert']
You should probably TRAIN this model on a down-stream task to be able to use it for pred

Epoch 1/7


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 50) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 50) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>]


1000/1000 [==============================] - ETA: 0s - loss: 1.5717 - accuracy: 0.2953

...

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 50) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>]


1000/1000 [==============================] - ETA: 0s - loss: 1.5625 - accuracy: 0.3021

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 50) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None,) dtype=int64>]


1000/1000 [==============================] - 171s 81ms/step - loss: 1.5625 - accuracy: 0.3021 - val_loss: 1.4736 - val_accuracy: 0.3313
Epoch 2/7
1000/1000 [==============================] - 70s 70ms/step - loss: 1.3152 - accuracy: 0.4209 - val_loss: 1.1861 - val_accuracy: 0.4935
Epoch 3/7
1000/1000 [==============================] - 71s 71ms/step - loss: 1.0828 - accuracy: 0.5427 - val_loss: 1.0470 - val_accuracy: 0.5776
Epoch 4/7
1000/1000 [==============================] - 71s 71ms/step - loss: 0.9063 - accuracy: 0.6371 - val_loss: 0.9582 - val_accuracy: 0.6356
Epoch 5/7
1000/1000 [==============================] - 71s 71ms/step - loss: 0.7814 - accuracy: 0.6975 - val_loss: 0.8864 - val_accuracy: 0.6730
Epoch 6/7
1000/1000 [==============================] - 71s 71ms/step - loss: 0.6908 - accuracy: 0.7375 - val_loss: 0.8732 - val_accuracy: 0.6885
Epoch 7/7
1000/1000 [==============================] - 71s 71ms/step - loss: 0.6246 - accuracy: 0.7667 - val_loss: 0.8652 - val_accuracy: 0

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 50) dtype=int64>]


In [84]:
test_results=np.mean(test_preds,axis=0)
result=np.argmax(test_results,axis=1)

In [89]:
from sklearn.metrics import  accuracy_score
accuracy_score(y_true=Y_test,y_pred=result)

0.697988061021446